In [1]:
"""
Extraction d'entités (slots) depuis les commandes utilisateur
Plusieurs approches : Regex, SpaCy NER, et modèle seq2seq
"""

import re
import json
from typing import Dict, Any, List, Optional
import torch
from transformers import (
    CamembertTokenizerFast,  # Changé de CamembertTokenizer à CamembertTokenizerFast
    CamembertForTokenClassification,
    pipeline
)
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, AutoModelForCausalLM

#Todo si erreur (windows ) : pip install "numpy<2"
# pip install transformers[torch] sentencepiece protobuf
# pip install --upgrade transformers torch


/home/lgerard/python/hermes/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


# ============================================================================
# APPROCHE 1: Règles et Regex (Simple et efficace pour cas simples)
# ============================================================================

class RuleBasedSlotExtractor:
    """Extracteur basé sur des règles pour chaque intention"""
    
    def __init__(self):
        self.patterns = {
            'export_notation': [
                (r'notation[s]?\s+(?:numéro\s+)?(\d+)', 'notation_id'),
                (r'n°\s*(\d+)', 'notation_id'),
                (r'la\s+(\d+)', 'notation_id'),
                (r'(?:^|\s)(\d+)(?:$|\s)', 'notation_id'),  # nombre seul
            ],
            'creer_notation': [
                (r'notation\s+"([^"]+)"', 'notation_nom'),
                (r'notation\s+(.+?)(?:\s+pour|\s+sur|$)', 'notation_nom'),
            ],
            'creer_essai': [
                (r'essai\s+"([^"]+)"', 'essai_nom'),
                (r'essai\s+(.+?)(?:\s+pour|\s+sur|$)', 'essai_nom'),
            ]
        }
    
    def extract(self, text: str, intent: str) -> Dict[str, Any]:
        """Extrait les slots pour une intention donnée"""
        slots = {}
        
        if intent not in self.patterns:
            return slots
        
        for pattern, slot_name in self.patterns[intent]:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                value = match.group(1)
                slots[slot_name] = value.strip()
                break  # Prend le premier match
        
        return slots


In [2]:
from transformers import (
    CamembertTokenizerFast, 
    CamembertForTokenClassification,
    Trainer,
    TrainingArguments,
    DataCollatorForTokenClassification,
    pipeline
)
from datasets import Dataset
from typing import List, Dict, Optional
import torch

class NERSlotExtractor:
    """
    Extraction de slots avec un modèle NER (Named Entity Recognition)
    Utilise BIO tagging: B-notation_id, I-notation_id, O
    """
    
    def __init__(self, model_path: Optional[str] = None):
        # Labels BIO pour chaque type d'entité
        self.label_list = [
            'O',
            'B-notation_id',
            'B-essai_id',
            'B-nom',
            'I-nom',
        ]
        
        self.label2id = {label: i for i, label in enumerate(self.label_list)}
        self.id2label = {i: label for i, label in enumerate(self.label_list)}
        
        # Charger le tokenizer
        if model_path:
            self.tokenizer = CamembertTokenizerFast.from_pretrained(model_path)
            self.model = CamembertForTokenClassification.from_pretrained(model_path)
        else:
            self.tokenizer = CamembertTokenizerFast.from_pretrained('camembert-base')
            self.model = CamembertForTokenClassification.from_pretrained(
                'camembert-base',
                num_labels=len(self.label_list),
                id2label=self.id2label,
                label2id=self.label2id
            )
        
        # Pipeline NER (sera créé après l'entraînement)
        self.ner_pipeline = None
    

    def create_training_data(self):
        """Crée des données d'entraînement annotées au format BIO - VERSION AUGMENTÉE"""
        examples = [
            # Variations pour notation_id
            ("export de la notation 123", [(22, 25, 'notation_id')]),
            ("exporter notation 456", [(18, 21, 'notation_id')]),
            ("je veux exporter la notation 789", [(29, 32, 'notation_id')]),
            ("je veux exporter la notation numéro 789", [(37, 40, 'notation_id')]),
            ("export notation 12", [(16, 18, 'notation_id')]),
            ("exporte la notation 999", [(19, 22, 'notation_id')]),
            ("notation 555 à exporter", [(9, 12, 'notation_id')]),
            ("exporter la notation 1", [(21, 22, 'notation_id')]),
            ("je souhaite exporter la notation 234", [(33, 36, 'notation_id')]),
            ("peux-tu exporter la notation 567", [(29, 32, 'notation_id')]),
            ("export la notation 890", [(19, 22, 'notation_id')]),
            ("notation 111", [(9, 12, 'notation_id')]),
            ("la notation 222", [(12, 15, 'notation_id')]),
            ("voir la notation 333", [(17, 20, 'notation_id')]),
            ("afficher notation 444", [(18, 21, 'notation_id')]),
            ("récupérer notation 555", [(19, 22, 'notation_id')]),
            ("télécharger notation 666", [(21, 24, 'notation_id')]),
            ("exporte notation 777", [(17, 20, 'notation_id')]),
            ("extraire notation 888", [(18, 21, 'notation_id')]),
            ("sortir notation 999", [(16, 19, 'notation_id')]),
            
            # Variations pour essai_id
            ("essai 777", [(6, 9, 'essai_id')]),
            ("export de l'essai 888", [(18, 21, 'essai_id')]),
            ("exporter l'essai 999", [(17, 20, 'essai_id')]),
            ("je veux l'essai 111", [(16, 19, 'essai_id')]),
            ("voir essai 222", [(11, 14, 'essai_id')]),
            ("essai numéro 333", [(13, 16, 'essai_id')]),
            ("récupérer essai 444", [(16, 19, 'essai_id')]),
            ("l'essai 555", [(8, 11, 'essai_id')]),
            
            # Exemples sans entités (important !)
            ("créer une notation", []),
            ("nouvelle notation", []),
            ("je veux créer", []),
            ("exporter", []),
            ("afficher tout", []),
            ("liste des notations", []),
            ("voir les essais", []),
            ("bonjour", []),
            ("merci", []),
            ("aide", []),
        ]
        
        tokenized_examples = []
        
        for text, entities in examples:
            encoding = self.tokenizer(
                text, 
                return_offsets_mapping=True, 
                truncation=True,
                padding='max_length',
                max_length=128
            )
            
            tokens = self.tokenizer.convert_ids_to_tokens(encoding['input_ids'])
            offsets = encoding['offset_mapping']
            
            labels = [self.label2id['O']] * len(tokens)
            
            for start, end, entity_type in entities:
                for idx, (token_start, token_end) in enumerate(offsets):
                    if token_start is None:
                        labels[idx] = -100
                        continue
                    
                    if token_start >= start and token_end <= end:
                        if token_start == start:
                            labels[idx] = self.label2id[f'B-{entity_type}']
                        else:
                            labels[idx] = self.label2id.get(f'I-{entity_type}', 
                                                            self.label2id[f'B-{entity_type}'])
            
            tokenized_examples.append({
                'input_ids': encoding['input_ids'],
                'attention_mask': encoding['attention_mask'],
                'labels': labels
            })
        
        return tokenized_examples
    

    def train(
        self, 
        output_dir: str = "./models/ner-notation",
        num_epochs: int = 50,  # Beaucoup plus d'époques
        batch_size: int = 4,   # Batch plus petit
        learning_rate: float = 2e-5,  # Learning rate plus petit
        save_model: bool = True
    ):
        """Entraîne le modèle NER sur les données annotées"""
        print("📚 Préparation des données d'entraînement...")
        training_data = self.create_training_data()
        
        print(f"   Nombre d'exemples: {len(training_data)}")
        
        dataset = Dataset.from_list(training_data)
        
        data_collator = DataCollatorForTokenClassification(
            tokenizer=self.tokenizer,
            padding=True
        )
        
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=num_epochs,
            per_device_train_batch_size=batch_size,
            learning_rate=learning_rate,
            weight_decay=0.01,
            logging_dir=f'{output_dir}/logs',
            logging_steps=5,
            save_strategy="epoch",
            eval_strategy="no",
            warmup_steps=100,  # Warmup pour stabiliser l'entraînement
            save_total_limit=2,  # Garde seulement les 2 derniers checkpoints
        )
        
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=dataset,
            tokenizer=self.tokenizer,
            data_collator=data_collator,
        )
        
        print("🚀 Début de l'entraînement...")
        train_result = trainer.train()
        
        print(f"\n📊 Résultats de l'entraînement:")
        print(f"   Loss finale: {train_result.training_loss:.4f}")
        
        if save_model:
            print(f"💾 Sauvegarde du modèle dans {output_dir}")
            self.model.save_pretrained(output_dir)
            self.tokenizer.save_pretrained(output_dir)
        
        self._create_pipeline()
        
        print("✅ Entraînement terminé !")


    
    def _create_pipeline(self):
        """Crée le pipeline NER pour l'inférence"""
        self.ner_pipeline = pipeline(
            "ner",
            model=self.model,
            tokenizer=self.tokenizer,
            aggregation_strategy="simple",
            device=0 if torch.cuda.is_available() else -1
        )
    
    def extract_slots(self, text: str) -> Dict[str, str]:
        """
        Extrait les slots d'une phrase
        
        Args:
            text: La phrase à analyser (ex: "je veux exporter la notation 34")
        
        Returns:
            Dictionnaire des slots (ex: {'notation_id': '34'})
        """
        if self.ner_pipeline is None:
            self._create_pipeline()
        
        # Prédiction NER
        entities = self.ner_pipeline(text)
        
        # Conversion en dictionnaire de slots
        slots = {}
        for entity in entities:
            # Nettoie le label (enlève B- ou I- si présent)
            slot_name = entity['entity_group'].replace('B-', '').replace('I-', '')
            slot_value = entity['word'].strip()
            
            # Si le slot existe déjà, concatène (pour gérer les multi-tokens)
            if slot_name in slots:
                slots[slot_name] += ' ' + slot_value
            else:
                slots[slot_name] = slot_value
        
        return slots
    
    @classmethod
    def load_trained_model(cls, model_path: str):
        """
        Charge un modèle déjà entraîné
        
        Args:
            model_path: Chemin vers le modèle sauvegardé
        
        Returns:
            Instance de NERSlotExtractor avec le modèle chargé
        """
        print(f"📥 Chargement du modèle depuis {model_path}")
        return cls(model_path=model_path)


# ============================================================================
# UTILISATION
# ============================================================================

if __name__ == "__main__":
    # Scénario 1: Entraîner un nouveau modèle
    print("=" * 60)
    print("SCÉNARIO 1: Entraînement d'un nouveau modèle")
    print("=" * 60)
    
    extractor = NERSlotExtractor()
    extractor.train(
        output_dir="./models/ner-notation",
        num_epochs=20,  # Plus d'époques pour un petit dataset
        batch_size=4,
        learning_rate=2e-5
    )
    
    # Tester l'extraction
    test_phrases = [
        "je veux exporter la notation 34",
        "export notation 789",
        "exporter l'essai 555",
        "créer une nouvelle notation test"
    ]
    
    print("\n" + "=" * 60)
    print("TESTS D'EXTRACTION")
    print("=" * 60)
    
    for phrase in test_phrases:
        slots = extractor.extract_slots(phrase)
        print(f"\n📝 Phrase: {phrase}")
        print(f"🎯 Slots: {slots}")
    
    print("\n" + "=" * 60)
    print("SCÉNARIO 2: Charger un modèle existant")
    print("=" * 60)
    
    # Charger le modèle entraîné
    extractor_loaded = NERSlotExtractor.load_trained_model("./models/ner-notation")
    
    # Tester
    result = extractor_loaded.extract_slots("je veux exporter la notation 99")
    print(f"\n📝 Phrase: je veux exporter la notation 99")
    print(f"🎯 Slots: {result}")

SCÉNARIO 1: Entraînement d'un nouveau modèle


Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/lgerard/python/hermes/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:827: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/tmp/ipykernel_7590/3170723008.py:170: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


📚 Préparation des données d'entraînement...
   Nombre d'exemples: 38
🚀 Début de l'entraînement...


/home/lgerard/python/hermes/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
5,1.664800
10,1.641000
15,1.616000
20,1.579000
25,1.533400
30,1.487400
35,1.417800
40,1.330500
45,1.270200
50,1.191000


...


📊 Résultats de l'entraînement:
   Loss finale: 0.6909
💾 Sauvegarde du modèle dans ./models/ner-notation


Device set to use cpu


✅ Entraînement terminé !

TESTS D'EXTRACTION

📝 Phrase: je veux exporter la notation 34
🎯 Slots: {}

📝 Phrase: export notation 789
🎯 Slots: {}

📝 Phrase: exporter l'essai 555
🎯 Slots: {}


Device set to use cpu



📝 Phrase: créer une nouvelle notation test
🎯 Slots: {}

SCÉNARIO 2: Charger un modèle existant
📥 Chargement du modèle depuis ./models/ner-notation

📝 Phrase: je veux exporter la notation 99
🎯 Slots: {}


In [3]:
extractor = NERSlotExtractor()
extractor.create_training_data()

[{'text': 'export de la notation 123',
  'tokens': ['<s>', '▁export', '▁de', '▁la', '▁notation', '▁123', '</s>'],
  'labels': ['O', 'O', 'O', 'O', 'O', 'B-notation_id', 'O'],
  'input_ids': [5, 13056, 8, 13, 18231, 18634, 6],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1]},
 {'text': 'exporter notation 456',
  'tokens': ['<s>', '▁export', 'er', '▁notation', '▁4', '56', '</s>'],
  'labels': ['O', 'O', 'O', 'O', 'B-notation_id', 'I-notation_id', 'O'],
  'input_ids': [5, 13056, 108, 18231, 181, 4876, 6],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1]},
 {'text': 'je veux exporter la notation numéro 789',
  'tokens': ['<s>',
   '▁je',
   '▁veux',
   '▁export',
   'er',
   '▁la',
   '▁notation',
   '▁numéro',
   '▁7',
   '89',
   '</s>'],
  'labels': ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-notation_id',
   'O'],
  'input_ids': [5, 50, 920, 13056, 108, 13, 18231, 1124, 333, 4283, 6],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'text': 'export notation 

In [4]:
import os 
# ============================================================================
# APPROCHE 3: Template avec LLM (Plus puissant mais plus coûteux)
# ============================================================================

class MistralSlotExtractor:
    """
    Extracteur de slots utilisant Mistral-7B-Instruct en local
    Optimisé pour l'extraction d'informations structurées
    """
    
    def __init__(
        self, 
        model_name: str = "mistralai/Mistral-7B-Instruct-v0.2",
        local_model_path: str = "../models/mistral-7b-instruct",  # Nouveau paramètre
        device: str = "auto",
        load_in_8bit: bool = False,
        load_in_4bit: bool = True
    ):

        """
        Initialise le modèle Mistral
        
        Args:
            model_name: Nom du modèle sur HuggingFace
            device: Device à utiliser ('cuda', 'cpu', ou 'auto')
            load_in_8bit: Charger en quantification 8-bit (nécessite bitsandbytes)
            load_in_4bit: Charger en quantification 4-bit (recommandé, économise de la VRAM)
        """
           
        # Vérifier si le modèle existe déjà en local
        if os.path.exists(local_model_path):
            print(f"🔄 Chargement du modèle depuis {local_model_path}...")
            model_name = local_model_path
        else:
            print(f"🔄 Chargement du modèle {model_name} depuis HuggingFace...")
        
        print(torch.cuda.is_available())
        self.device = device if device != "auto" else ("cuda" if torch.cuda.is_available() else "cpu")

    
        
        # Configuration de quantification pour économiser la mémoire
        kwargs = {}
        if load_in_4bit and self.device == "cuda":
            kwargs["load_in_4bit"] = True
            kwargs["device_map"] = "auto"
            print("   Using 4-bit quantization")
        elif load_in_8bit and self.device == "cuda":
            kwargs["load_in_8bit"] = True
            kwargs["device_map"] = "auto"
            print("   Using 8-bit quantization")
        else:
            kwargs["torch_dtype"] = torch.float16 if self.device == "cuda" else torch.float32
        
        # Chargement
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name, **kwargs)
        
        if not load_in_4bit and not load_in_8bit:
            self.model = self.model.to(self.device)
        
        self.model.eval()
        
        # Sauvegarder en local si ce n'est pas déjà fait
        if not os.path.exists(local_model_path):
            print(f"💾 Sauvegarde du modèle dans {local_model_path}...")
            os.makedirs(local_model_path, exist_ok=True)
            self.tokenizer.save_pretrained(local_model_path)
            self.model.save_pretrained(local_model_path)
            print("✓ Modèle sauvegardé")
        
        print(f"✓ Modèle chargé sur {self.device}")
        
        # Définition des templates pour chaque intention
        self.templates = {
            'export_notation': {
                'slots': ['notation_id'],
                'description': 'notation_id est le numéro/identifiant de la notation à exporter (nombre uniquement)'
            },
            'creer_notation': {
                'slots': ['nom', 'description'],
                'description': 'nom est le nom de la nouvelle notation, description est une description optionnelle'
            },
            'creer_essai': {
                'slots': ['nom', 'type'],
                'description': "nom est le nom du nouvel essai, type est le type d'essai (optionnel)"
            },
            'modifier_essai': {
                'slots': ['essai_id', 'champ', 'valeur'],
                'description': 'essai_id est l\'identifiant de l\'essai, champ est le champ à modifier, valeur est la nouvelle valeur'
            }
        }
    
    def build_prompt(self, text: str, intent: str) -> str:
        """
        Construit le prompt pour Mistral au format Instruct
        """
        template_info = self.templates.get(intent, {})
        slots = template_info.get('slots', [])
        description = template_info.get('description', '')
        
        # Construction du JSON de sortie attendu
        json_schema = {slot: "valeur ou null" for slot in slots}
        json_example = json.dumps(json_schema, ensure_ascii=False, indent=2)
        
        # Prompt optimisé pour l'extraction
        user_message = f"""Extrait les informations structurées de la phrase suivante.

Phrase: "{text}"
Intention: {intent}

Informations à extraire:
{description}

Réponds UNIQUEMENT avec un objet JSON valide, sans texte avant ou après.
Format attendu:
{json_example}

Si une information n'est pas présente dans la phrase, utilise null."""

        return user_message
    
    def format_mistral_prompt(self, user_message: str) -> str:
        """
        Formate le prompt au format Mistral Instruct
        """
        return f"<s>[INST] {user_message} [/INST]"
    
    def generate(self, prompt: str, max_new_tokens: int = 150, temperature: float = 0.1) -> str:
        """
        Génère une réponse avec le modèle Mistral
        
        Args:
            prompt: Le prompt formaté
            max_new_tokens: Nombre maximum de tokens à générer
            temperature: Température de génération (plus bas = plus déterministe)
        """
        # Tokenisation
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        
        # Génération
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=temperature > 0,
                top_p=0.95,
                pad_token_id=self.tokenizer.eos_token_id
            )
        
        # Décodage (enlève le prompt)
        generated_text = self.tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
        return generated_text.strip()
    
    def extract_json_from_response(self, response: str) -> Optional[Dict]:
        """
        Extrait un objet JSON de la réponse (gère les cas où le modèle ajoute du texte)
        """
        # Méthode 1: Chercher un bloc JSON avec regex
        json_pattern = r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}'
        matches = re.findall(json_pattern, response, re.DOTALL)
        
        for match in matches:
            try:
                return json.loads(match)
            except json.JSONDecodeError:
                continue
        
        # Méthode 2: Essayer de parser directement
        try:
            return json.loads(response)
        except json.JSONDecodeError:
            pass
        
        # Méthode 3: Nettoyer et réessayer
        cleaned = response.strip()
        if cleaned.startswith('```json'):
            cleaned = cleaned[7:]
        if cleaned.startswith('```'):
            cleaned = cleaned[3:]
        if cleaned.endswith('```'):
            cleaned = cleaned[:-3]
        
        try:
            return json.loads(cleaned.strip())
        except json.JSONDecodeError:
            return None
    
    def extract(self, text: str, intent: str, verbose: bool = False) -> Dict[str, Any]:
        """
        Extrait les slots depuis le texte pour l'intention donnée
        
        Args:
            text: La phrase utilisateur
            intent: L'intention détectée
            verbose: Affiche les détails de génération
        
        Returns:
            Dictionnaire des slots extraits
        """
        # Construction du prompt
        user_message = self.build_prompt(text, intent)
        prompt = self.format_mistral_prompt(user_message)
        
        if verbose:
            print(f"\n📝 Prompt envoyé au modèle:")
            print(f"{user_message}\n")
        
        # Génération
        response = self.generate(prompt)
        
        if verbose:
            print(f"🤖 Réponse brute du modèle:")
            print(f"{response}\n")
        
        # Extraction du JSON
        slots = self.extract_json_from_response(response)
        
        if slots is None:
            if verbose:
                print("⚠️  Échec de l'extraction JSON, retour d'un dict vide")
            return {}
        
        # Nettoyage des valeurs null
        slots = {k: v for k, v in slots.items() if v is not None and v != "null"}
        
        if verbose:
            print(f"✓ Slots extraits: {slots}")
        
        return slots
    
    def extract_batch(self, texts_and_intents: list, verbose: bool = False) -> list:
        """
        Extrait les slots pour plusieurs phrases en batch
        
        Args:
            texts_and_intents: Liste de tuples (text, intent)
            verbose: Affiche les détails
        
        Returns:
            Liste de dictionnaires de slots
        """
        results = []
        for text, intent in texts_and_intents:
            if verbose:
                print(f"\n{'='*70}")
                print(f"Traitement: '{text}' (intent: {intent})")
                print('='*70)
            
            slots = self.extract(text, intent, verbose=verbose)
            results.append(slots)
        
        return results



In [5]:

# ============================================================================
# APPROCHE 4: Système hybride (Recommandé)
# ============================================================================

class HybridSlotExtractor:
    """
    Combine plusieurs approches:
    1. Essaie d'abord les règles (rapide, précis pour cas simples)
    2. Si insuffisant, utilise NER ou LLM
    """
    
    def __init__(self):
        self.rule_extractor = RuleBasedSlotExtractor()
        # self.ner_extractor = NERSlotExtractor()  # Si modèle entraîné
        self.llm_extractor = MistralSlotExtractor(device="cpu")
    
    def extract(self, text: str, intent: str, use_llm: bool = False) -> Dict[str, Any]:
        """
        Extrait les slots en utilisant la meilleure approche
        
        Args:
            text: La phrase utilisateur
            intent: L'intention détectée
            use_llm: Si True, utilise le LLM pour les cas complexes
        """
        # Essaie d'abord les règles
        slots = self.rule_extractor.extract(text, intent)
        
        # Si on a trouvé des slots, c'est bon
        if slots:
            return {'method': 'rules', 'slots': slots, 'confidence': 'high'}
        
        # Sinon, utilise le LLM si demandé
        if use_llm:
            slots = self.llm_extractor.extract(text, intent)
            return {'method': 'llm', 'slots': slots, 'confidence': 'medium'}
        
        return {'method': 'none', 'slots': {}, 'confidence': 'low'}



In [7]:

# ============================================================================
# DÉMONSTRATION
# ============================================================================

def demo():
    """Démonstration des différentes approches"""
    
    print("="*70)
    print("DÉMONSTRATION D'EXTRACTION DE SLOTS")
    print("="*70)
    
    test_cases = [
        ("export de la notation 345", "export_notation"),
        ("je veux exporter la notation numéro 789", "export_notation"),
        ("exporter n° 123", "export_notation"),
        ("créer une notation analyse des risques", "creer_notation"),
        ("nouveau essai de résistance", "creer_essai"),
    ]
    
    # Approche 1: Règles
    print("\n🔧 APPROCHE 1: RÈGLES ET REGEX")
    print("-" * 70)
    rule_extractor = RuleBasedSlotExtractor()
    
    for text, intent in test_cases:
        slots = rule_extractor.extract(text, intent)
        print(f"\n📝 '{text}'")
        print(f"   Intent: {intent}")
        print(f"   Slots: {slots}")
    
    # Approche 2: NER
    print("\n\n🤖 APPROCHE 2: NER (Named Entity Recognition)")
    print("-" * 70)
    print("Nécessite un modèle entraîné sur des données annotées BIO")
    ner_extractor = NERSlotExtractor()
    training_data = ner_extractor.create_training_data()
    print(f"Exemple de données d'entraînement ({len(training_data)} exemples):")
    for i, example in enumerate(training_data[:2]):
        print(f"\n  Exemple {i+1}:")
        print(f"    Texte: {example['text']}")
        print(f"    Tokens: {example['tokens'][:10]}...")
        print(f"    Labels: {example['labels'][:10]}...")
    
    # Approche 3: LLM
    print("\n\n🧠 APPROCHE 3: LLM (Large Language Model)")
    print("-" * 70)
    llm_extractor = MistralSlotExtractor()
    
    for text, intent in test_cases[:3]:
        slots = llm_extractor.extract(text, intent)
        print(f"\n📝 '{text}'")
        print(f"   Intent: {intent}")
        print(f"   Slots: {slots}")
    
    # Approche 4: Hybride
    print("\n\n⚡ APPROCHE 4: SYSTÈME HYBRIDE (RECOMMANDÉ)")
    print("-" * 70)
    hybrid_extractor = HybridSlotExtractor()
    
    for text, intent in test_cases:
        result = hybrid_extractor.extract(text, intent)
        print(f"\n📝 '{text}'")
        print(f"   Intent: {intent}")
        print(f"   Méthode: {result['method']}")
        print(f"   Confiance: {result['confidence']}")
        print(f"   Slots: {result['slots']}")
    
    # Recommandations
    print("\n\n💡 RECOMMANDATIONS")
    print("="*70)
    print("""
1. RÈGLES + REGEX (Recommandé pour commencer):
   ✓ Simple à implémenter et maintenir
   ✓ Très rapide (< 1ms)
   ✓ Précis pour les patterns connus
   ✓ Pas besoin d'entraînement
   ✗ Limité aux patterns prédéfinis
   
2. NER (Pour cas plus complexes):
   ✓ Plus flexible que les règles
   ✓ Peut gérer des variations
   ✓ Bonne précision avec peu de données (100-200 exemples)
   ✗ Nécessite annotation des données
   ✗ Temps d'entraînement
   
3. LLM (Pour flexibilité maximale):
   ✓ Très flexible, comprend le contexte
   ✓ Pas besoin d'entraînement
   ✓ Gère les cas complexes
   ✗ Coût API ou ressources GPU
   ✗ Latence plus élevée (100-1000ms)
   ✗ Moins prédictible
   
4. HYBRIDE (Meilleur compromis):
   ✓ Combine avantages de chaque approche
   ✓ Règles pour cas simples (rapide)
   ✓ LLM pour cas complexes (flexible)
   ✓ Optimise coût/performance
   """)

if __name__ == "__main__":
    demo()

print("end")

DÉMONSTRATION D'EXTRACTION DE SLOTS

🔧 APPROCHE 1: RÈGLES ET REGEX
----------------------------------------------------------------------

📝 'export de la notation 345'
   Intent: export_notation
   Slots: {'notation_id': '345'}

📝 'je veux exporter la notation numéro 789'
   Intent: export_notation
   Slots: {'notation_id': '789'}

📝 'exporter n° 123'
   Intent: export_notation
   Slots: {'notation_id': '123'}

📝 'créer une notation analyse des risques'
   Intent: creer_notation
   Slots: {'notation_nom': 'analyse des risques'}

📝 'nouveau essai de résistance'
   Intent: creer_essai
   Slots: {'essai_nom': 'de résistance'}


🤖 APPROCHE 2: NER (Named Entity Recognition)
----------------------------------------------------------------------
Nécessite un modèle entraîné sur des données annotées BIO
Exemple de données d'entraînement (7 exemples):

  Exemple 1:
    Texte: export de la notation 123
    Tokens: ['<s>', '▁export', '▁de', '▁la', '▁notation', '▁123', '</s>']...
    Labels: ['O'

Loading checkpoint shards: 100%|██████████| 3/3 [03:07<00:00, 62.40s/it]


✓ Modèle chargé sur cpu

📝 'export de la notation 345'
   Intent: export_notation
   Slots: {'notation_id': '345'}

📝 'je veux exporter la notation numéro 789'
   Intent: export_notation
   Slots: {'notation_id': '789'}

📝 'exporter n° 123'
   Intent: export_notation
   Slots: {'notation_id': '123'}


⚡ APPROCHE 4: SYSTÈME HYBRIDE (RECOMMANDÉ)
----------------------------------------------------------------------
🔄 Chargement du modèle mistralai/Mistral-7B-Instruct-v0.2...


Loading checkpoint shards: 100%|██████████| 3/3 [02:03<00:00, 41.02s/it]


✓ Modèle chargé sur cpu

📝 'export de la notation 345'
   Intent: export_notation
   Méthode: rules
   Confiance: high
   Slots: {'notation_id': '345'}

📝 'je veux exporter la notation numéro 789'
   Intent: export_notation
   Méthode: rules
   Confiance: high
   Slots: {'notation_id': '789'}

📝 'exporter n° 123'
   Intent: export_notation
   Méthode: rules
   Confiance: high
   Slots: {'notation_id': '123'}

📝 'créer une notation analyse des risques'
   Intent: creer_notation
   Méthode: rules
   Confiance: high
   Slots: {'notation_nom': 'analyse des risques'}

📝 'nouveau essai de résistance'
   Intent: creer_essai
   Méthode: rules
   Confiance: high
   Slots: {'essai_nom': 'de résistance'}


💡 RECOMMANDATIONS

1. RÈGLES + REGEX (Recommandé pour commencer):
   ✓ Simple à implémenter et maintenir
   ✓ Très rapide (< 1ms)
   ✓ Précis pour les patterns connus
   ✓ Pas besoin d'entraînement
   ✗ Limité aux patterns prédéfinis
   
2. NER (Pour cas plus complexes):
   ✓ Plus flexible que 

In [6]:
hybrid_extractor = HybridSlotExtractor()

result = hybrid_extractor.extract("Je voudrais exporter les résultats de la notation 415", "export_notation")
result

🔄 Chargement du modèle depuis ../models/mistral-7b-instruct...
False


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 6/6 [00:00<00:00, 54.78it/s]

✓ Modèle chargé sur cpu


{'method': 'rules', 'slots': {'notation_id': '415'}, 'confidence': 'high'}

In [12]:
result = hybrid_extractor.extract("Je voudrais exporter la notation 567", "export_notation", use_llm=True)
result

{'method': 'rules', 'slots': {'notation_id': '567'}, 'confidence': 'high'}